---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction -- Marlon

All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [1]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [2]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [3]:
def answer_one():
    return df.where(df['Gold'] >= df['Gold'].max()).dropna() # Karl
    return df[df['Gold'] == max(df['Gold'])].index[0] # Wale
    return (df[df['Gold']==df['Gold'].max()])['ID'] # GP
    return df['Gold'].idxmax() # Emmanuel

answer_one()

# !!! Google question: how to find index of column maximum
# https://thispointer.com/pandas-find-maximum-values-position-in-columns-or-rows-of-a-dataframe/
# skipna=False

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [9]:
# !!!
print(abs(df['Gold'] - df['Gold.1']))
(abs(df['Gold'] - df['Gold.1'])).idxmax() # skipna=True is default

Afghanistan                          0
Algeria                              5
Argentina                           18
Armenia                              1
Australasia                          3
                                    ..
Yugoslavia                          26
Independent Olympic Participants     0
Zambia                               0
Zimbabwe                             3
Mixed team                           8
Length: 146, dtype: int64


'United States'

In [10]:
print(type(abs(df['Gold'] - df['Gold.1'])))

<class 'pandas.core.series.Series'>


In [11]:
def answer_two():
    df['Diff_Gold_Su_Wi'] = abs(df['Gold'] - df['Gold.1'])
    return  df.where(df['Diff_Gold_Su_Wi'] >=df['Diff_Gold_Su_Wi'].max()).dropna().index[0]  # Karl

#     return df[df['Gold'] - df['Gold.1'] == max(df['Gold'] - df['Gold.1'])].index[0] # Wale

#     df2 = pd.DataFrame()
#     df2['difference'] = abs(df['Gold'] - df['Gold.1'])
#     res = df2[df2['difference']==df2['difference'].max()]
#     return res                                                     # GP

#     summer = df['Gold']
#     winter = df['Gold.1']
#     delt = summer - winter
#     return delt[delt == max(delt)].index[0]                        # Emmanuel

answer_two()
# df['Gold']-df

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [21]:
df1 = df[df['Gold.2']>0] #[['Gold', 'Gold.1', 'Gold.2']]
( abs(df1['Gold'] - df1['Gold.1']) / df1['Gold.2'] ).idxmax()
#!!! ???

'Algeria'

In [32]:
df.loc[['Algeria','Bulgaria']]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Bulgaria,19,51,85,78,214,19,1,2,3,6,38,52,87,81,220,BUL


In [33]:
print(5/5, 50/52)

1.0 0.9615384615384616


In [25]:
def answer_three():
     res1 = df.where(df['Gold'] != 0)
     res2 = res1.where(df['Gold.1'] != 0)
     res2['Diff_Gold_Su_Wi_Rel'] = abs(res2['Gold'] - res2['Gold.1']) / res2['Gold.2']
     res = res2.where(res2['Diff_Gold_Su_Wi_Rel'] >= res2['Diff_Gold_Su_Wi_Rel'].max()).dropna()
     k = res.index[0]                                     # Karl

     df_copy = df[(df['Gold'] > 0) & (df['Gold.1'] > 0)]
     res = df_copy[(df_copy['Gold']-df_copy['Gold.1'])/df_copy['Gold.2'] == max((df_copy['Gold']-df_copy['Gold.1'])/df_copy['Gold.2'])]
     w = res.index[0]                            # Wale

     df2 = pd.DataFrame()
     df2['difference'] = (abs(df['Gold'] - df['Gold.1']))/(df['Gold.2'])  # if (df['Gold.2'] > 0) else 0
     res = df2[df2['difference']==df2['difference'].max()]
     gp =  res                                     # GP

     f1 = df[df['Gold']>0]
     f2 = f1[f1['Gold.1']>0]
     summer = f2['Gold']
     winter = f2['Gold.1']
     total = summer + winter
     relative = (summer - winter) / total
     e =  relative[relative == max(relative)].index[0]            # Emmanuel
     return k, w, gp, e

answer_three()


('Bulgaria',
 'Bulgaria',
              difference
 Algeria             1.0
 Argentina           1.0
 Armenia             1.0
 Australasia         1.0
 Azerbaijan          1.0
 ...                 ...
 Uruguay             1.0
 Venezuela           1.0
 Yugoslavia          1.0
 Zimbabwe            1.0
 Mixed team          1.0
 
 [64 rows x 1 columns],
 'Bulgaria')

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [30]:
points = 3*df['Gold.2'] + 2*df['Silver.2'] + 1*df['Bronze.2']

In [24]:
def answer_four():
#     df['Points'] = df['Gold.2']*3 + df['Silver.2']*2 + df['Bronze.2']*1
#     return df['Points']        # Karl

#     res = df['Gold.2']*3 + df['Silver.2']*2 + df['Bronze.2']
#     return pd.Series(res.values, index = res.index.values)  # Wale

#     df2 = pd.DataFrame()
#     df2['points'] = (df['Gold.2'] * 3) +  ( df['Silver.2'] * 2 ) + df['Bronze.2']
#     return df2['points']  # GP

#     Points = df['Gold.2'] * 3 + df['Silver.2'] * 2 + df['Bronze.2'] * 1
#     return Points   # Emmanuel

answer_four()

Afghanistan                           2
Algeria                              27
Argentina                           130
Armenia                              16
Australasia                          22
                                   ... 
Yugoslavia                          171
Independent Olympic Participants      4
Zambia                                3
Zimbabwe                             18
Mixed team                           38
Length: 146, dtype: int64

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)
*This function should return a single string value.*

SUMLEV column: SUMLEV=40 refers to the population aggregates for each state, while SUMLEV=50 refers to the population for each county.  (That's why STNAME=Alabama and CTYNAME=Alabama in the first row where SUMLEV=50).
Therefore, in your calculations for Q5-Q8 (and in especially Q6 and Q7) you need to get the rows with SUMLEV equal to 50.




In [34]:
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [87]:
# filter by SUMLEV=50
df = census_df[census_df['SUMLEV']==50]

In [91]:
# Solution without groupby
x = 0; stateWithMaxCounties = ''
for state in census_df['STNAME'].unique():
    noCounties = df[df['STNAME']==state]['CTYNAME'].unique().shape[0]
    if noCounties > x:
        x = noCounties
        stateWithMaxCounties=state
        
stateWithMaxCounties

'Texas'

In [88]:
# Solution with groupby 
# count - number of elements
df.groupby('STNAME')['CTYNAME'].count().sort_values(ascending = False).index[0]

'Texas'

In [89]:
# Solution with groupby
# nunique - number of unique elements
df.groupby('STNAME')['CTYNAME'].nunique().sort_values(ascending = False).index[0]

'Texas'

In [84]:
def answer_five():
    grb = census_df.groupby(['STNAME']).size().reset_index(name='counts')
    k = grb.where(grb['counts'] >= grb['counts'].max()).dropna()['STNAME']       # Karl

    res = census_df.groupby('STNAME').count()[['SUMLEV']]
    w = res[res['SUMLEV'] == max(res['SUMLEV'])].index[0]                        # Wale
    
    return k, w

answer_five()


(43    Texas
 Name: STNAME, dtype: object,
 'Texas')

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [172]:
# Solution without groupby
res = []

for state in df['STNAME'].unique():
    df1 = df[df['STNAME']==state]
    df1 = df[df['STNAME']==state].sort_values(by=['CENSUS2010POP'], ascending = False)
    res.append([state, df1['CENSUS2010POP'].head(3).sum()])
    
df_res = pd.DataFrame(res, columns = ['STNAME', 'TOP3POP'])
df_res.sort_values(by=['TOP3POP'], ascending = False)['STNAME'].head(3).tolist()
#df_res.nlargest(3,'TOP3POP')['STNAME'].tolist()

['California', 'Texas', 'Illinois']

In [173]:
# Solution with groupby
# Google search: "sum top 3 in a group groupby" 
# https://stackoverflow.com/questions/40390634/pandas-groupby-nlargest-sum
df.groupby('STNAME')['CENSUS2010POP'].nlargest(3).sum(level = 0).nlargest(3).index.tolist()

['California', 'Texas', 'Illinois']

In [174]:
def answer_six():
    min_df = census_df[census_df['SUMLEV']==50]
    min_df = min_df[['STNAME', 'CENSUS2010POP']]    
    min_df = min_df.sort_values(by=['CENSUS2010POP'], ascending=False)
    min_df = min_df.groupby('STNAME').head(3).groupby('STNAME').sum()
    min_df = min_df.sort_values(by=['CENSUS2010POP'], ascending=False)
    w = min_df[:3].index.values.tolist()                 # Wale

    f1 = census_df[census_df['SUMLEV'] == 50].groupby('STNAME')['CENSUS2010POP'].apply(lambda x: x.nlargest(3).sum()).nlargest(3).index.values.tolist()
    e = f1# Emmanuel
    
    return e, w

answer_six()

(['California', 'Texas', 'Illinois'], ['California', 'Texas', 'Illinois'])

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [136]:
print(df.columns)

Index(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME',
       'CENSUS2010POP', 'ESTIMATESBASE2010', 'POPESTIMATE2010',
       'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013',
       'POPESTIMATE2014', 'POPESTIMATE2015', 'NPOPCHG_2010', 'NPOPCHG_2011',
       'NPOPCHG_2012', 'NPOPCHG_2013', 'NPOPCHG_2014', 'NPOPCHG_2015',
       'BIRTHS2010', 'BIRTHS2011', 'BIRTHS2012', 'BIRTHS2013', 'BIRTHS2014',
       'BIRTHS2015', 'DEATHS2010', 'DEATHS2011', 'DEATHS2012', 'DEATHS2013',
       'DEATHS2014', 'DEATHS2015', 'NATURALINC2010', 'NATURALINC2011',
       'NATURALINC2012', 'NATURALINC2013', 'NATURALINC2014', 'NATURALINC2015',
       'INTERNATIONALMIG2010', 'INTERNATIONALMIG2011', 'INTERNATIONALMIG2012',
       'INTERNATIONALMIG2013', 'INTERNATIONALMIG2014', 'INTERNATIONALMIG2015',
       'DOMESTICMIG2010', 'DOMESTICMIG2011', 'DOMESTICMIG2012',
       'DOMESTICMIG2013', 'DOMESTICMIG2014', 'DOMESTICMIG2015', 'NETMIG2010',
       'NETMIG2011', 'NETMIG2012', 'NETMI

In [272]:
# Solution 1 
abs_change_max = 0
for index, row in df.loc[:,['STNAME','CTYNAME']].iterrows():
    df1 = df[(df['CTYNAME']==row['CTYNAME']) & (df['STNAME']==row['STNAME'])]
    s = df1.loc[:,'POPESTIMATE2010':'POPESTIMATE2015'].iloc[0]
    abs_change = s.max() - s.min()
    if abs_change > abs_change_max:
        abs_change_max = abs_change
        state_max = row['STNAME']
        county_max = row['CTYNAME']

county_max + ', ' + state_max


'Harris County, Texas'

In [273]:
#Solution 2
df1 = df.loc[:,'POPESTIMATE2010':'POPESTIMATE2015']
idx = (df1.max(axis =1) - df1.min(axis=1)).idxmax()
res = df.loc[idx,['STNAME','CTYNAME']].tolist()
', '.join(res)

'Texas, Harris County'

__Conclusion__: Do not iterate over dataframe !!!

In [271]:
def answer_seven():
    min_df = census_df[census_df['SUMLEV']==50]
    min_df = min_df[['CTYNAME', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']]
    min_df["diff"] = min_df.max(axis=1, numeric_only=True) - min_df.min(axis=1, numeric_only=True)
    min_df = min_df[min_df["diff"] == max(min_df["diff"])]
    w = min_df.iloc[0].CTYNAME                     # Wale

    f1 = census_df[census_df['SUMLEV'] == 50].set_index(['STNAME','CTYNAME']).loc[:,['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].stack()
    f2 = f1.max(level=['STNAME','CTYNAME']) - f1.min(level=['STNAME','CTYNAME'])
    e = f2.idxmax()[1]                             # Emmanuel

    return w, e

answer_seven()

('Harris County', 'Harris County')

In [270]:
df[df['CTYNAME']=='Harris County'].loc[:,'POPESTIMATE2010':'POPESTIMATE2015']


,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
469,32157,32359,32607,32658,32873,33381
2667,4108187,4181238,4262504,4352462,4447577,4538028


### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [162]:
def answer_eight():
#     res = census_df[((census_df['REGION']==1) | (census_df['REGION']==2)) & (census_df['POPESTIMATE2015']>census_df['POPESTIMATE2014']) & (census_df['CTYNAME'].str.startswith('Washington'))]
#     return res[['STNAME', 'CTYNAME']]                               # Wale

    f1 = census_df[(census_df['POPESTIMATE2015'] > census_df['POPESTIMATE2014']) & ((census_df['REGION'] == 1) | (census_df['REGION'] == 2))]
    f1 = f1[f1['CTYNAME'].str.startswith('Washington')]
    return f1.loc[:,['STNAME', 'CTYNAME']]                          # Emmanuel

answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
